In [32]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
# !conda install -c conda-forgefolium=0.5.0 --yes
# !pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [42]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [43]:
url= 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
source=requests.get(url).text
soup=BeautifulSoup(source, 'lxml')
print(soup.title)
from IPython.display import display_html
tab=str(soup.table)
dfs=pd.read_html(tab)
df=dfs[0]
print(df.shape)
df.head()

<title>List of postal codes of Canada: M - Wikipedia</title>
(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [44]:
# drop rows where Borough is not assigned
df1=df[df.Borough != 'Not assigned']
# combine neigborhoods with same postal codes
df=df1.groupby(['Postal Code', 'Borough'], sort=False).agg(','.join)
df.reset_index(inplace=True)
# replacing the name of the neighbourhoods which are not assgined Borough
df['Neighbourhood']=np.where(df1['Neighbourhood']=='Not assigned', df1['Borough'], df1['Neighbourhood'])
df.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
print(df.shape)
df.head()

(103, 3)


,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#Get Longitutde and Latitude of each Neighbourhoods

In [45]:
import geocoder
lat_lng=pd.read_csv('https://cocl.us/Geospatial_data')
print(lat_lng.shape)
lat_lng.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [46]:
lat_lng.rename(columns={'Postal Code': 'Postalcode'}, inplace=True)
df1=pd.merge(df, lat_lng, on='Postalcode')
df1.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Explore and Cluster Neighbourhoods in Toronto

In [47]:
df1= df1[df1['Borough'].str.contains('Toronto', regex=False)]
print(df1.shape)
df1.head()

(40, 5)


,Postalcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [48]:
map_toronto=folium.Map(location=[43.651070, -79.347015], zoom_start=10)
for lat, lng, borough, neighbourhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
        label= '{}, {}'.format(neighbourhood, borough)
        label=folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.5, parse_html=False).add_to(map_toronto)
map_toronto

In [49]:
from sklearn.cluster import KMeans
k=7
toronto_clustring=df1.drop(['Postalcode', 'Borough', 'Neighbourhood'], 1)
toronto_clustring


,Latitude,Longitude
2,43.654260,-79.360636
4,43.662301,-79.389494
9,43.657162,-79.378937
15,43.651494,-79.375418
19,43.676357,-79.293031
20,43.644771,-79.373306
24,43.657952,-79.387383
25,43.669542,-79.422564
30,43.650571,-79.384568
31,43.669005,-79.442259


In [50]:
Kmeans=KMeans(n_clusters=k, random_state=0).fit(toronto_clustring)
Kmeans.labels_
df1.insert(0, 'Cluster', Kmeans.labels_)
df1

,Cluster,Postalcode,Borough,Neighbourhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,3,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,0,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,4,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,4,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [52]:
clusters_map=folium.Map(location=[43.651070, -79.347015], zoom_start=10)
#set color scheme for each cluster
x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]
# add circlemarker to the map
marker_colors=[]
for lat, lng, neighbourhood, cluster in zip(df1['Latitude'], df1['Longitude'], df1['Neighbourhood'], df1['Cluster']):
    label=folium.Popup('Cluster'+str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=2,
    popup=label, 
    color=rainbow[cluster-1],
    fill=True,
    fill_color= rainbow[cluster-1],
    fill_opacity=0.7).add_to(clusters_map)

In [53]:
clusters_map